# Evaluacion e implementacion del Modelo
## Macro Average de F1-score
Usaremos con una metrica el ```macro average``` que se calcula como se muestra en la figura. De esta manera podremos usar una sola métrica para medir el rendimiento general del modelo

<img src="images/macro-avg-f1-score.png" class="center" alt="macro avg de f1-score" />

## Analisis usando ROC y AUC
- Modelos con mejor performance tendran más Lift, es decir, la curva se alejará más del centro
- Más lift significa que el score de AUC será mayor

## Calibracion del Modelo
Mediremos que tan bien calibradas están las predicciones del modelo.

Lo que esto significa es que queremos que las probabilidades de  ```Default``` representen correctamente el nivel de confianza del modelo
- La probabilidad de ```Default``` tiene un grado de incertidumbre en su prediccion
- Una muestra de prestamos y sus probabilidades de ```default``` predecidas debe ser cercana al porcentage de defaults en esa muestra

<table style="width:100%; align:center; text-align: center">
  <tr>
    <th>Muestra de Prestamos</th>
    <th>Promedio Predicho de PD</th>
    <th>Porcentaje Real de Defaults en la Muestra</th>
    <th>Calibrado?</th>
  </tr>
  <tr>
    <td>10</td>
    <td>0.12</td>
    <td>0.12</td>
    <td>Si</td>
  </tr>
  <tr>
    <td>10</td>
    <td>0.25</td>
    <td>0.65</td>
    <td>No</td>
  </tr>
</table>

En la tabla de arriba de puede ver que en la primera fila para una muestra de 10 prestamos el promedio predicho de los prestamos como defaults es del 12%, el cual coincide con el porcentaje real. En cambio, en la segunda fila el promedio predicho de defaults es de 25%, pero en realidad el 65% de la muestra contiene prestamos con default, por lo que el modelo no está calibrado

Para calcular este valor utilizaremos la funcion ```calibration_curve```. El esta funcion toma como paremetros el test set, las probabildades y ```n_bins``` que representa el numero de muestras a tomar

Llamemos ```PD``` a las probababilidades predichas y ```%Default``` al porcentaje real de ```Defaults``` en la muestra. 
Supongamos que para una de las muestras tenemos los valores:
- ```PD=0.56```
- ```%Default=0.75```

Esto indica que estamos prediciendo menos defaults de los que deberíamos, esto indica varios ```falsos negativos```, lo cual puede afectar negativamente al negocio 

Si ahora para otra muestra tenemos los siguientes valores:
- ```PD=0.94```
- ```%Default=0.66```

En esa muestra encontramos la mayor parte de nuestros falsos positivos. Estas son oportunidades que se pueden perder al predecirlas como default. Pero para este problema resultan menos perjudiciales que los defaults

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_curve, \
                            classification_report, \
                            precision_recall_fscore_support, \
                            roc_auc_score, \
                            confusion_matrix
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import matplotlib
import xgboost as xgb
cr_loan_clean = pd.read_csv("data/cr_loan_nout_nmiss.csv")